# 6. The Laplacian Pyramid

In [568]:
%matplotlib nbagg
import warnings
import inspect
import matplotlib.pyplot as plt
import IPython.display
import numpy as np
from cued_sf2_lab.familiarisation import load_mat_img, plot_image

## 6.1 The basic concept

<figure id="figure-2">
<div style="background-color: white">

![](figures/laplacian.svg)</div>
    
<figcaption style="text-align: center">Figure 2: A typical laplacian pyramid with 2 levels; forward (analysis) part only</figcaption>
</figure>

The Laplacian pyramid is an energy compaction technique, based on
the observation:

> For most real-world images, the high-frequency energy is much less than the low-frequency energy.

Since the lowpass image is much lower bandwidth than the
original image, it can be subsampled (*decimated*) 2:1 in both
horizontal and vertical directions, without significant loss of
information to give a quarter-size lowpass image.  We have
provided a row filter/decimation function, `rowdec(X, h)`, to
filter and decimate the rows of a matrix by 2:1.

In [569]:
from cued_sf2_lab.laplacian_pyramid import rowdec

Use this twice
(on the image and its transpose) to generate a quarter-size
lowpass image `X1` of Lighthouse.  For simplicity use a 3-tap (length 3)
filter `h` with coefficients: $\frac{1}{4} [1\ 2\ 1]$.

In [570]:
X, cmaps_dict = load_mat_img(img='lighthouse.mat', img_info='X', cmap_info={'map', 'map2'})

In [571]:
h = 0.25*np.array([1, 2, 1])
X1 = rowdec(rowdec(X, h).T, h).T

fig, ax = plt.subplots()
plot_image(X1, ax=ax)

<IPython.core.display.Javascript object>

The image `X1` can be interpolated 2:1 in each direction to
generate a lowpass image of the original size, which can then be
subtracted from the original to yield a full-size highpass image.
We have also provided a row interpolation/filter function,
`rowint(X, h)`, which doubles the length of each row of the image, by
inserting zeros between alternate samples and then lowpass
filtering the result with the filter `h`. Note that to undo the decimation
performed by `Y = rowdec(X, h)`, this should be called as `Z = rowint(Y, 2*h)`,
where a DC gain of 2 is added, as shown in [Figure 2](#figure-2).

In [572]:
from cued_sf2_lab.laplacian_pyramid import rowint

Apply this twice (as before) to generate
a full-size lowpass image, and subtract this from `X` to
give a highpass image `Y0`. Display `X1` and `Y0` to see these effects.

In [573]:
Xl = rowint(rowint(X1, 2*h).T, 2*h).T

fig, ax = plt.subplots()
plot_image(Xl, ax=ax)

<IPython.core.display.Javascript object>

In [574]:
Y0 = X - Xl

fig, ax = plt.subplots()
plot_image(Y0, ax=ax)

<IPython.core.display.Javascript object>

The highpass images will
always have approximately zero mean (since any dc component is
removed). Hence for the remainder of this project, we recommend
that you also make all lowpass images be approximately zero mean
by subtracting 128 from them before you start any processing. This
makes the 8-bit pixel values cover the range -128 to 127,
instead of 0 to 255. The advantages of having a zero-mean input image are not very obvious here, but they
will become clearer as the project progresses. If you use `draw_image` the images will still be displayed correctly.

In [575]:
X, cmaps_dict = load_mat_img(img='lighthouse.mat', img_info='X', cmap_info={'map', 'map2'})
X = X - 128.0

Examine the functions ```rowdec``` and ```rowint``` and check that you understand how they work. Decimation is achieved by selecting every second element of a vector using selection indices `i:i+c:2`, and filtering is performed with symmetric extension as in ```convse```, except that we do not bother to calculate the filter outputs that are to be discarded by the decimation process. Interpolation is achieved by loading every second element of a double-size vector using `X2 = np.zeros((r, c2), dtype=X.dtype)`, `X2[:, ::2] = X`. The intermediate samples of the interpolated vector x must be zero before the vector is passed through the interpolation filter.

In [576]:
IPython.display.Code(inspect.getsource(rowdec), language="python")

def rowdec(X: np.ndarray, h: np.ndarray) -> np.ndarray:
    """
    Filter rows of image X with h and then decimate by a factor of 2.

    Parameters:
        X: Image matrix (Usually 256x256)
        h: Filter coefficients
    Returns:
        Y: Image with filtered and decimated rows

    If len(H) is odd, each output sample is aligned with the first of
    each pair of input samples.
    If len(H) is even, each output sample is aligned with the mid point
    of each pair of input samples.
    """
    r, c = X.shape
    m = len(h)
    m2 = m // 2
    if m % 2:
        X = np.pad(X, [(0, 0), (m2, m2)], mode='reflect')
    else:
        X = np.pad(X, [(0, 0), (m2-1, m2-1)], mode='symmetric')

    Y = np.zeros((r, (c+1)//2))
    # Loop for each term in h.
    for i in range(m):
        Y = Y + h[i] * X[:, i:i+c:2]
    return Y

In [577]:
IPython.display.Code(inspect.getsource(rowint), language="python")

def rowint(X: np.ndarray, h: np.ndarray) -> np.ndarray:
    """
    Interpolates the rows of image X by 2 using h.

    Parameters:
        X: Image matrix (Usually 256x256)
        h: Filter coefficients
    Returns:
        Y: Image with interpolated rows

    If len(h) is odd, each input sample is aligned with the first of
    each pair of output samples.
    If len(h) is even, each input sample is aligned with the mid point
    of each pair of output samples.
    """
    r, c = X.shape
    m = len(h)
    m2 = m // 2
    c2 = 2 * c

    # Generate X2 as X interleaved with columns of zeros.
    X2 = np.zeros((r, c2), dtype=X.dtype)
    X2[:, ::2] = X

    X2 = np.pad(X2, [(0, 0), (m2, m2)], mode='reflect' if m % 2 else 'symmetric')

    Y = np.zeros((r, c2))
    # Loop for each term in h.
    for i in range(m):
        Y = Y + h[i] * X2[:, i:i+c2]
    return Y

If the small lowpass image `X1` and the full-size highpass image `Y0`
are transmitted to a distant decoder, then the decoder can exactly reconstruct
the original image by interpolating `X1` up to full size and adding in
`Y0` (which represents the error between the original and the interpolated
`X1`).  We have achieved image compression if `X1` and `Y0` can be
transmitted with fewer bits than `X`.  Usually this will be the case
because `Y0` contains so much less energy than `X`, and `X1` is
only one quarter of the size of `X`.  However we do start at a
disadvantage because there are 25% more samples to code.  Many of the `Y0` samples may be represented by zero, and we shall show later that runs of
zeros may be coded with relatively few bits.

The quarter-size lowpass image `X1` may be further subsampled, using the
same process as was applied to `X`, so that it may be transmitted as a
one-sixteenth-size lowpass image `X2` and a quarter-size highpass image
`Y1`.  This usually achieves further data compression and may be repeated
as many times as is desired (until, for typical images, no further compression
is achieved).  This leads to a pyramid of highpass images and a final tiny
lowpass image.  Usually three or four layers of the pyramid are sufficient to
give maximum compression.

Write a `py4enc(X, h)` function to generate a 4-layer pyramid, so that `X` is split into four
highpass images, `Y0 Y1 Y2 Y3`, each a quarter of the size of its predecessor, plus a tiny
lowpass image `X4`, which is a quarter of the size of `Y3`.

In [578]:
def py4enc(X, h):

    X1 = rowdec(rowdec(X, h).T, h).T
    Y0 = X - rowint(rowint(X1, 2*h).T, 2*h).T
    X2 = rowdec(rowdec(X1, h).T, h).T
    Y1 = X1 - rowint(rowint(X2, 2*h).T, 2*h).T
    X3 = rowdec(rowdec(X2, h).T, h).T
    Y2 = X2 - rowint(rowint(X3, 2*h).T, 2*h).T
    X4 = rowdec(rowdec(X3, h).T, h).T
    Y3 = X3 - rowint(rowint(X4, 2*h).T, 2*h).T

    return Y0, Y1, Y2, Y3, X4

In [579]:
# General function for n layer encoding

def pynenc(X, h, n):

    Xn = X
    Ys = []

    for _ in range(n):

        # Decimate X 2:1
        Xd = rowdec(rowdec(Xn, h).T, h).T

        # Interpolate Xd to upscale and subtract from Xn to obtain Y
        Ys.append(Xn - rowint(rowint(Xd, 2*h).T, 2*h).T)

        Xn = Xd

    return Ys, Xn



We can then plot the results to check them using the `beside` helper function:

In [580]:
from cued_sf2_lab.laplacian_pyramid import beside

In [581]:
Y0, Y1, Y2, Y3, X4 = py4enc(X, h)
fig, ax = plt.subplots(figsize=(8, 4))
plot_image(beside(Y0, beside(Y1, beside(Y2, beside(Y3, X4)))), ax=ax);

<IPython.core.display.Javascript object>

If we wish to see the images separately from each other, instead of using our `beside` function that was written to match the old Matlab version, we can draw this directly with `matplotlib` using [`plt.subplots`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html) to create a set of subplots, and the `width_ratios` argument to [`GridSpec`](https://matplotlib.org/stable/api/_as_gen/matplotlib.gridspec.GridSpec.html):

In [582]:
titles = ["Y_0", "Y_1", "Y_2", "Y_3", "X_4"]
imgs = [Y0, Y1, Y2, Y3, X4]
fig, axs = plt.subplots(1, 5, figsize=(8, 4),
                        gridspec_kw=dict(width_ratios=[img.shape[0] for img in imgs]))

for ax, img, title in zip(axs, imgs, titles):
    plot_image(img, ax=ax)
    ax.set(yticks=[], title=f'${title}$')

<IPython.core.display.Javascript object>

If we wish to see the images at the same scale as each other, we can use:

In [583]:
titles = ["Y_0", "Y_1", "Y_2", "Y_3", "X_4"]
imgs = [Y0, Y1, Y2, Y3, X4]
fig, axs = plt.subplots(1, 5, figsize=(8, 2))

for ax, img, title in zip(axs, imgs, titles):
    plot_image(img, ax=ax)
    ax.set(yticks=[], title=f'${title}$')

<IPython.core.display.Javascript object>

Get a demonstrator to check that your images look correct, and
then write another function `py4dec` to decode `X4` and
`Y3 Y2 Y1 Y0` into a set of lowpass images `Z3 Z2 Z1 Z0`.
(`Z3` is obtained by interpolating `X4` and adding
`Y3`, and then `Z2` is obtained from `Z3` and `Y2`, and
so on.)

In [584]:
def py4dec(Y0, Y1, Y2, Y3, X4, h):

    Z3 = Y3 + rowint(rowint(X4, 2*h).T, 2*h).T
    Z2 = Y2 + rowint(rowint(Z3, 2*h).T, 2*h).T
    Z1 = Y1 + rowint(rowint(Z2, 2*h).T, 2*h).T
    Z0 = Y0 + rowint(rowint(Z1, 2*h).T, 2*h).T

    return Z3, Z2, Z1, Z0

In [585]:
# N layer decoding

def pyndec(Ys, Xn, h):

    Zn = Xn
    Ysr = reversed(Ys)
    Zs = [Xn]

    for y in Ysr:

        # interpolate X to upscale and add to each y, for every y
        Zn = y + rowint(rowint(Zn, 2*h).T, 2*h).T
        Zs.append(Zn)

    return Zs


If all is correct, `Z0` should be identical to
`X`.  You can check that your function is correct by calculating `np.max(abs(X - Z0))` and also displaying your pyramid of decoded images, `Z3` to `Z0`.

In [586]:
h = 0.25*np.array([1, 2, 1])

Z3, Z2, Z1, Z0 = py4dec(Y0, Y1, Y2, Y3, X4, h)
Zs = pyndec([Y0, Y1, Y2, Y3], X4, h)

encode_decode_err = np.max(np.abs(X - Z0))
err_n = np.max(np.abs(X - Zs[-1]))
print(f'Encode-Decode Error = {encode_decode_err}')
print(f'Encode-Decode Error N layer function = {err_n}')

# your code here to plot the images

Encode-Decode Error = 0.0
Encode-Decode Error N layer function = 0.0


For further information on the Laplacian Pyramid see Burt and Adelson [IEEE Trans. on Communications, 1983, vol 31, no 4, pp 532-540, "The Laplacian Pyramid as a compact image code"].

## 6.2 Quantisation and Coding Efficiency
To see whether data compression is possible using the above pyramid decomposition, we must calculate the approximate number of bits required to code the image pyramid. This may be done using the *entropy* of the quantised image data. 

The entropy of a single data sample, whcih may randomly take one of $Q$ possible quantised values such that each value $q(i)$ has a probability $p(i)$ of being in state $i$ for $i=1\to Q$ is given by:

$$ \text{Entropy (bits/sample)} = \sum_{i=1}^Qp(i) \log_2\frac{1}{p(i)} = -\sum_{i=1}^Q p(i) \log_2p(i)$$

The entropy represents the minimum average number of bits per sample needed to code samples with the given probability distribution $p(i)$, assuming that an ideal variable-length entropy code is used, and that the samples are uncorrelated with each other. Arithmetic codes can get arbitrarily close to this bit rate, and simpler Huffman codes can also get vary close with many typical signals (you will be using Huffman codes later on). It is possible to code signals at bit rates less than the entropy, if the samples are correlated, but for simplicity we shall ignore this here.

To demonstrate the validity of the above formula, first consider a signal with 8 quantised values of equal probability $p(i) = 1/8$ for all $i$. The entropy is then $8 \times 1/8 \times \log_2(8) = 3$ bits per sample, as expected. 

Now consider a signal with only 3 values, with probabilities $p(1)=1/2$,  $p(2) = p(3) = 1/4$. The entropy is then $\frac{1}{2} \log_2(2) + 2 \times \frac{1}{4} \log_2(4) = 1.5$ bits per sample. This is consistent with using a single bit '0' to represent state 1 and two bits, '10' and '11' to represent states 2 and 3.

The function `bpp` has been written to calculate the entropy in bits per pixel of an image matrix `X`. First the function computes a histogram of `X` to determine the probabilities $p(i)$ and then it calculates the entropy, using the above formula.

In [587]:
from cued_sf2_lab.laplacian_pyramid import bpp

In the Laplacian Pyramid, the total number of bits is obtained by multiplying each of the sub-image entropies by the number of pixels in each corresponding sub-image. However, in order to compress the data, we also need to quantise the images. We have also provided a function `quantise` which will quantise `X` in steps centred on integer multiples of `step`. Hence `bpp(quantise(X, step))` will return the entropy of image `X` quantised in steps of `step`.

In [588]:
from cued_sf2_lab.laplacian_pyramid import quantise

<div class="alert alert-block alert-danger">

Calculate the entropies of images `X` `X1` `Y0` and hence the total numbers of bits to encode `X`, or `X1` and `Y0`, when quantised to a step size of 17 (which gives 15 distict grey levels if applied to a lowpass image with intensities from -127 to 127. Find the data compression for this simple one-stage pyramid, and then investigate the improvements from using more layers.
</div>

In [614]:
sX = bpp(quantise(X, 17))*(256**2)
sX1 = bpp(quantise(Z0, 17))*(128**2)
sY0 = bpp(quantise(Y0, 17))*(256**2)
print("Bits to encode X in bits", sX)
print("Bits to encode X1 in bits", sX1)
print("Bits to encode Y0 in bits", sY0)

Bits to encode X in bits 228119.03651868744
Bits to encode X1 in bits 57029.75912967186
Bits to encode Y0 in bits 106223.16163959922


Since compressing an image will generally result in a reduction in quality, we also need a way to measure this quality reduction. It is actually quite hard to find a quality measure whcih matches individual perceptions of how an image has been changed due to compression, and for that reason it is important to always judge and comment on an image visually. However we also need a quantitative measure, and the most obvious is the rms error (standard deviation) between the input and compressed image (i.e. using ```np.std(X - Z)``` where `Z` is the compressed image).

<div class="alert alert-block alert-danger">
Quantise the Laplacian pyramid with a step size of 17, and reconstruct the output image from the decoding pyramid. Look at the visual features, and calculate the rms error (standard deviation) between the input image and the decoding pyramid output image. Repeat this for more layers in the pyramid.
</div>

In [615]:
# For 4 layers, using the py4enc and py4dec functions

_, _, _, Z0q = py4dec(quantise(Y0, 17), quantise(Y1, 17), quantise(Y2, 17), quantise(Y3, 17), quantise(X4, 17), h)

print("RMS Error for 4 layer quantised", np.std(X - Z0q))
fig, ax = plt.subplots(1, 2)

plot_image(quantise(X, 17), ax=ax[0])
plot_image(Z0q, ax=ax[1])

# See more general results below

RMS Error for 4 layer quantised 7.629817744060247


<IPython.core.display.Javascript object>

In [591]:
# For n layers

steps = 17

fig, ax = plt.subplots(1, 5, figsize=(16, 5))

Xq = quantise(X, steps)
plot_image(Xq, ax=ax[0])
ax[0].set(yticks=[], title="Quantised")
print("Error for quantised ", np.std(X - Xq))

for i, n in enumerate([1, 2, 3, 4]):

    # Encode with n layers in the pyramid
    Ys, Xn, = pynenc(X, h, n)

    # Quantise layers
    Ysq = []
    for y in Ys:
        Ysq.append(quantise(y, steps))

    Xnq = quantise(Xn, steps)

    # Decode
    Zsq = pyndec(Ysq, Xnq, h)

    # Print errors and show plots
    print("Error for " + str(n+1) + " layers ", np.std(X - Zsq[-1]))
    plot_image(Zsq[-1], ax=ax[i + 1])
    ax[i+1].set(yticks=[], title=f'${n+1}$ Layers')


<IPython.core.display.Javascript object>

Error for quantised  4.861168497356846
Error for 2 layers  5.382782204619935
Error for 3 layers  6.067419036722591
Error for 4 layers  6.751868881610953
Error for 5 layers  7.629817744060247


Note that we call this error the rms error, but in fact we calculate the standard deviation, which only equals the true rms error if the mean error is zero. However the eye is very insensitive to small errors in the mean level of images, so the standard deviation (which ignores the mean) is a better measure of image quality. 

<div class="alert alert-block alert-danger">
Quantise the original image with the same step size (17) and note the visual features and rms error. Compare these results from the pyramid scheme above. Why are the rms errors larger in the pyramid scheme? 
</div>

In [592]:
# See above

Comparisons of the number of bits with different coding strategies are only valid if they result in approximately the same image quantisation error. Write a function which will optimise the step size (resulting in a non-integer value) in the Laplacian scheme until the rms error is the same as for direct quantisation; you will find this optimisation useful for later investigations too.
<div class="alert alert-block alert-danger">
Investigate what step size of the quantisers for the pyramid scheme you need, in order to get approximately the same error as for direct quantisation at a step size of 17.
</div>

In [689]:
# Varying step size for 4 layer pyramid

Xq = quantise(X, 17)
print("Error for direct 17 step quantised ", np.std(X - Xq))

for steps in [10, 10.3, 17, 26, 50]:

    # Encode with 4 layers in the pyramid
    Ys, Xn, = pynenc(X, h, 4)

    # Quantise layers
    Ysq = []
    for y in Ys:
        Ysq.append(quantise(y, steps))

    Xnq = quantise(Xn, steps)

    # Decode
    Zsq = pyndec(Ysq, Xnq, h)

    # Print errors
    print("Error for 5 layer " + str(steps) + " steps ", np.std(X - Zsq[-1]))

Error for direct 17 step quantised  4.861168497356846
Error for 5 layer 10 steps  4.704559666362862
Error for 5 layer 10.3 steps  4.859984210471377
Error for 5 layer 17 steps  7.629817744060247
Error for 5 layer 26 steps  10.883856286167628
Error for 5 layer 50 steps  20.53969987097942


Optimum step size = 10.3 for 4 layer pyramid

In many of your results from now on you will need to expres the performance of your algorithms in terms of *compression ratio*, which is normally defined as:

$$ \text{Compression Ratio} = \frac{\text{Total bits for reference scheme}}{\text{Total bits for compressed scheme}} $$

Usually the *reference* scheme is the direct pixel quantisation method with its quantiser adjusted to give the same rms error as the scheme being evaluated (the *compressed* scheme). For good schemes we try to make the compression ratio as large as possible.

We now investigate the effect of using different step sizes for the different levels of the pyramid. There are many schemes for varying the step sizes between different levels: in this project we shall look at the *equal MSE* criterion. In the *equal MSE* scheme, step sizes are chosen such that quantisers in each layer contribute equally to the Mean Squared Error of the reconstructed image. In general the step sizes will depend on the image signal being coded. However this can be achieved approximately by choosing a separate step size for each layer such that:
* A single impulse of that step size will give a filtered pulse in the reconstructed image which has the same energy, whichever layer of the decoder the impulse excites.

### Impulse Response Measurement
Investigate the effect of a single impulse in a particular layer (e.g. **Y0,Y1** etc.) as it appears in the reconstructed image **Z0**. This can be done by first generating a test pyramid image, which is zero everywhere. Then place an impulse (e.g. of amplitude 100) in the centre of one layer, reconstruct the entire pyramid to give **Z0**, then measure the total energy of **Z0**. If this is repeated for each layer, you will have measured how much energy a fixed size impulse at each layer contributes to the decoded image.

We actually want to arrange for the impulse sizes to vary and the energy to stay the same. Therefore the impulse sizes (and hence chosen quantisation steps) we require in each level will be inversely proportional to the square root of the energies measured above. The important result is the *ratio* of the step sizes between layers. If this ratio is maintained for any overall quantiser scaling, we will get an (approximately) *equal MSE* scheme.

<div class="alert alert-block alert-danger">
Find new values for the data compression achievable when all schemes (i.e. constant step size or equal MSE, both with varying layer depth) produce the same rms error between the decoded image and the original image. Comment on the differences in compression, visual quality and rms error, and the optimum choice of layer depth in each case.
</div>

In [617]:
# Constant step size, for different pyramid heights

steps = 17

# Image plot and error for direct quantisation
fig, ax = plt.subplots(1, 5, figsize=(16, 5))
Xq = quantise(X, steps)
plot_image(Xq, ax=ax[0])
ax[0].set(yticks=[], title="Quantised")
print("Direct Quantised", "Compression Ratio 1, Error", np.std(X - Xq))

# Reference size for compression ratios
ref_size = bpp(Xq)*(256**2)

for i, n in enumerate([1, 2, 3, 4]):

    # Encode with n layers in the pyramid
    Ys, Xn = pynenc(X, h, n)

    # Quantise layers
    Ysq = []
    for y in Ys:
        Ysq.append(quantise(y, steps))

    Xnq = quantise(Xn, steps)

    # Calculate size of scheme
    size = 0
    # top of pyramid
    size += bpp(Xnq) * (256/(2**n))**2
    for k in range(n):
        # multiplying bits per pixel with no. pixels for each layer
        size += bpp(Ysq[k]) * (256/(2**k))**2

    # Decode
    Zsq = pyndec(Ysq, Xnq, h)

    # Print errors & compression ratio
    print(str(n+1), "layers", "Compression ratio", ref_size/size, ", Error", np.std(X - Zsq[-1]))

    # Show plots
    plot_image(Zsq[-1], ax=ax[i + 1])
    ax[i+1].set(yticks=[], title=f'${n+1}$ Layers')

<IPython.core.display.Javascript object>

Direct Quantised Compression Ratio 1, Error 4.861168497356846
2 layers Compression ratio 1.406814651240581 , Error 5.382782204619935
3 layers Compression ratio 1.599247333158242 , Error 6.067419036722591
4 layers Compression ratio 1.6498864463331373 , Error 6.751868881610953
5 layers Compression ratio 1.6608214118976627 , Error 7.629817744060247


<h1>Find the step size ratios for equal MSE<h1>

In [690]:
A = np.zeros((256, 256)) # Blank image
Ys, Xn = pynenc(A, h, 7) # Encoding to n layers

energies = []

# placing impulse on each layer then finding the energy of the decoded image
for i in range(len(Ys)):

    Ysp = Ys
    mid = int(256/(2**(i+1))) # midpoint for layer

    Ysp[i][mid][mid] = 100  # creating impulse

    Zs = pyndec(Ysp, Xn, h) # decoding

    # finding energy of reconstructed image and storing
    energies.append(np.sum(Zs[-1]**2))


for i in range(len(energies)-1):

    print("Energy ratio from layer", i+2, "to layer", i+3, "=", energies[i+1]/energies[i])


Energy ratio from layer 2 to layer 3 = 5.25
Energy ratio from layer 3 to layer 4 = 3.988095238095238
Energy ratio from layer 4 to layer 5 = 3.8992537313432836
Energy ratio from layer 5 to layer 6 = 3.9442583732057415
Energy ratio from layer 6 to layer 7 = 3.977621762600837
Energy ratio from layer 7 to layer 8 = 3.9922122613853053


<h1>Finding compression ratios and errors for varying pyramid height and intial quant step size, variable step size for each layer<h1>
<p>And hence optimising inital step size for each layer height?<p>

In [691]:

s0 = 10
r = 2 # sqrt(4), ratio found above

fig, ax = plt.subplots(4, 4, sharex=True, sharey=True, figsize=(12, 12))
fig.suptitle("Decoded image, for Laplacian pyramid encoding with equal MSE")
ref_size = bpp(X)*(256**2)

layers = [1, 2, 3, 4]
steps = [10, 13, 17, 25]

for i, n in enumerate(layers):

    for j, s0 in enumerate(steps):

        # Encode with n layers in the pyramid
        Ys, Xn, = pynenc(X, h, n)

        # Quantise layers, with steps in ratio r between layers
        Ysq = []
        for k, y in enumerate(Ys):
            Ysq.append(quantise(y, s0/(r**k)))

        Xnq = quantise(Xn, s0/(r**len(Ys)))

        size = 0 # Calculate size of scheme
        size += bpp(Xnq) * (256/(2**n))**2 # top of pyramid
        for k in range(n):
            # multiplying bits per pixel with no. pixels for each layer
            size += bpp(Ysq[k]) * (256/(2**k))**2

        Zsq = pyndec(Ysq, Xnq, h) # Decode

        # Show plots
        plot_image(Zsq[-1], ax=ax[i][j])
        ax[i][j].set(yticks=[], title="CR: " + str(round(ref_size/size, 3)) + " , Err: " + str(round(float(np.std(X - Zsq[-1])), 3)))

        if j == 0:
            ax[i][j].set(ylabel=str(n + 1) + " layers")

        if i == len(layers)-1:
            ax[i][j].set(xlabel="s0 = " + str(s0))


<IPython.core.display.Javascript object>

In [599]:
# Find the quantisation step size for equal mse, matching the reference RMS error

#  Reference to what? there's only error if you quantise the original image, how to pick ref quant in this case?

In [600]:
# Find the compression ratios for equal mse

# Above

## 6.3 Changing the decimation / interpolation filter
It is also worth investigating whether a more complicated decimation / interpolation filter **h** can improve the compression. The z-transfer function of the filter used so far is

$$ h(z) = \left(\frac{1 + z^{-1}}{2}\right)^m $$

where $m = 2$. If $m$ is increased to 4(so the number of taps remains odd), the vector representation of **h** is \[1/16 4/16 6/16 4/16 1/16\]. This filter has a lower cut-off frequency than when $m=2$, so you should find that each interpolated lowpass image in the pyramid is a little more blurred, and there is a little more energy left in each highpass image.

<div class="alert alert-block alert-danger">
Optimise the step sizes for the pyramid with this new filter and determine the new compression performance and visual features.
</div>

In [601]:
# Repeat the previous experiments with a new h here
# h = np.array([1, 4, 6, 4, 1]) / 16.0

# First Interim Report
Discuss and explain your results, gathered so far, in your first interim report. Try to answer the questions posed in the above test. Be brief where things are straightforward, but pay more attention to detail in areas where you think something interesting is happening. Write this as a standalone report, not as a series of bullet points answering the questions posed.